# The MRI Signal Equation

The MRI signal equation can be used to understand how images are formed as well as the influence of other factors on contrast, image quality and artifacts.  It is derived from the Bloch equation and can incorporate the effects of RF coils and other in vivo spin physics.
The signal equation can be simplified in many ways, which we will use throughout this book.  Here we build up a comprehensive MRI signal equation. 

## Learning Goals

1. Understand what MRI is measuring
   * Describe what produces the MRI signal
   * Describe how gradients influence MRI signal
   * Describe how in vivo spin physics phenomena influence MRI signal

## Build

(ALL in rotating frame)

signal is from the transverse magnetization

(m shorthand for Mxy)

signal comes from the entire volume
actually from sensitive volume of the RF receive coil elements

signal decays
signal may experience off-resonance/chemical shift

signal may experience rotation due to magnetic field gradients

SIMPLIFIED VERSION

imaging - gradients only

But for now simplify


signal


voxel-wise signal after image reconstruciton (gradients get removed, more on that later)?  or in SE chapter?

## Idealized Signal Equation

Combining the effects of magnetic field gradients and the principles of the MRI signal, we obtain the following idealized signal equation:

$$s(t) = \int m(\vec{r}) \  \exp\left( -i \gamma \int_0^t \vec{G}(\tau) \cdot \vec{r} \ d\tau \right) \  d\vec{r}$$

using the notation $m(\vec{r}) = M_{XY}(\vec{r}, t=0)$ for the initial transverse magnetization for reasons that will become clear soon.

In many situations, an idealized signal equation is useful, particularly for understanding image formation.  The idealized signal equation neglects relaxation, RF coil profile, and off-resonance, but we will add them in later. 

This is simplified using the concept of k-space:

$$s(t) = \int m(\vec{r})\  e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r}$$

From this idealized formulation, it is easiest to see that the signal is the Fourier Transform of the transverse magnetization in the subject:

$$s(t) = \mathcal{F}\{m(\vec{r}) \} |_{\vec{k} = \vec{k}(t)}$$

Simplified using the notation $M(\vec{k}) = \mathcal{F}\{m(\vec{r}) \}$ as

$$s(t) = M(\vec{k}(t))$$

The is a key result in image formation, showing that our signal represents the Fourier Transform, meaning that an inverse Fourier Transform will be used for image reconstruction from MRI signals.

## Adding Coil Sensitivity

From the idealized signal equation, the RF coil receive profile simply modulates the transverse magnetization, and the received signal for coil element $n$, which has a coil sensitivity profile $C_n(\vec{r})$, is

$$s_n(t) = \int m(\vec{r})\ C_n(\vec{r})\ e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r}$$

And the signal is once again the Fourier Transform of the object, but now multiplied by the coil sensitivity:

$$s_n(t) = \mathcal{F}\{m(\vec{r}) \ C_n(\vec{r}) \} |_{\vec{k} = \vec{k}(t)}$$

## Relaxation during signal acquisition

Starting with the idealized signal equation, we can add in relaxation, which modulates the receieved signal over time.  This will create some type of weighting in k-space, which leads to image filtering (often blurring).  This weighting is spatially varying, depending on relaxation of different tissues.

<!-- $$s(t) = \int m(\vec{r}) \ e^{-t/T_2^*(\vec{r})}\  \exp\left( -i \gamma \int_0^t \vec{G}(\tau) \cdot \vec{r} d\tau \right) \  d\vec{r}$$ -->

$$s(t) = \int m(\vec{r})\ e^{-t/T_2^*(\vec{r})}\  e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r}$$

To interpret the effect of relaxation, consider a single value of $T_2^*(\vec{r}) = T_2^*$, in which case this can be removed from the integral and 

$$s(t) = e^{-t/T_2^*} \int m(\vec{r})\ e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r} \\
 = M(\vec{k}(t))\  e^{-t/T_2^*} $$

From this, we can see that relaxation leads to signal amplitude modulation as we acquire data across k-space.  To see the effect on image formation, we determine the function $\upsilon (\vec{k})$ which captures across k-space trajectory in time and across all TRs to characterize this modulation:

$$\hat{M}(\vec{k}) = M(\vec{k}) e^{- \upsilon(\vec{k}) / T_2^*}$$

Examples of $\upsilon(\vec{k})$ are shown below

## Off-resonance and Chemical Shift

Off-resonance and chemical shift lead to changes in the precession frequency of the magnetization.  We can add this onto the idealized signal equation.

Here we separately define:
* $\Delta f_r(\vec{r}) = \bar \gamma \Delta B_0(\vec{r})$ - magnetic field inhomogeneities, including main field imperfections, and magnetic susceptibility effects
* $\Delta f_{cs}$ - frequency changes due to chemical shift, so depends on the chemical species being examined (could sum or index signal over this dimension)


<!-- $$s(t) = \int m(\vec{r})\ e^{-i 2 \pi \Delta f_{cs} t} e^{-i 2 \pi \Delta f_r(\vec{r}) t} e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r}$$ -->
For simplicity, combine all off-resonance and/or chemical shift into a single frequnecy shift term, $\Delta f(\vec{r}) = \Delta f_{cs} + \Delta f_r(\vec{r})$ as

$$s(t) = \int m(\vec{r})\ e^{-i 2 \pi \Delta f(\vec{r}) t} e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r}$$

Solving for the effect of frequency shifts then requires specific knowledge of the k-space trajectory, and the interaction of a spatially varying field further complicates this effect.  However, some intuition can be gained by assuming a single frequency shift, $\Delta f(\vec{r}) = \Delta f_0$, which can then be removed from the integral as

$$s(t) = e^{-i 2 \pi \Delta f_0 t} \int m(\vec{r})\ e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r} \\
 = M(\vec{k}(t))\  e^{-i 2 \pi \Delta f_0 t} $$

From this, we can see that the frequency shift leads to a phase accumulation as we acquire data across k-space.  To see the effect on image formation, we determine the function $\upsilon (\vec{k})$ which captures across k-space trajectory in time and across all TRs to characterize this phase accumulation:

$$\hat{M}(\vec{k}) = M(\vec{k}) e^{-i 2 \pi\ \upsilon(\vec{k}) \Delta f_0}$$

Thus the reconstructed image will corrupted by a convolution based on the k-space phase accumulation

$$\hat{m}(\vec{r}) = \mathcal{F}^{-1} \{ \hat{M}(\vec{k})\} = m(\vec{r}) * \mathcal{F}^{-1} \{ e^{-i 2 \pi\ \upsilon(\vec{k}) \Delta f_0} \}$$

This convolution results in a shift for Cartesian and echo-planar sampling, and blurring for spiral and radial sampling.

This analysis can also be applied to look at the effect from off-resonance at a single location, $\vec{r} = \vec{r}_0$
$$s_{r_0}(t) = \mathcal{F}\{m(\vec{r}_0) \} |_{\vec{k} = \vec{k}(t)}  e^{-i 2 \pi \Delta f(\vec{r}_0) t} $$

$$\hat{m}(\vec{r}_0) = m(\vec{r}_0) \star \mathcal{F}^{-1} \{ e^{-i 2 \pi\ \upsilon(\vec{k}) \Delta f(\vec{r}_0)} \}$$


## Complete Signal Equation

Finally, we can write the most intimidating version, a full version of signal equation, including relaxation, coil sensitivity, off-resonance and chemical shift, and gradients.
The signal from an individual RF coil element $n$ is

$$s_n(t) = \int m(\vec{r})\ C_n(\vec{r})\ e^{-t/T_2^*(\vec{r})}\ e^{-i 2 \pi \Delta f_{cs} t} e^{-i 2 \pi \Delta f_r(\vec{r}) t} \  \exp\left( -i \gamma \int_0^t \vec{G}(\tau) \cdot \vec{r} \ d\tau \right) \  d\vec{r}$$

which is typically simplified for understanding image formation using k-space:

$$s_n(t) = \int m(\vec{r})\ C_n(\vec{r})\ e^{-t/T_2^*(\vec{r})}\ e^{-i 2 \pi \Delta f_{cs} t} e^{-i 2 \pi \Delta f_r(\vec{r}) t} \  e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r}$$

Note that this describes the signal behavior between RF pulses.  Incorporating RF pulses can be done either through rotation matrices or some solution of the Bloch equation such as numerical solvers or phase graphs. 